# Milestone 3 - Report
------
In this notebook we are going to review everything that was done in the project following Milestone 2. The focus at this point of the project was placed on constructing the awareness model and the final datastory 

In [41]:
import os
import json
import time
import folium
import branca
import jenkspy
import pickle
import unicodedata
import collections
import numpy as np
import pandas as pd

## 0. Abstract

*What's the motivation behind your project? A 150 word description of the project idea, goals, dataset used. What story you would like to tell and why?*

Major events happen on a regular basis all around the world, some involving high number of casualties but the resulting reaction on the international scale is often far from proportional. Most of the time the largest reaction comes from the place where the incident occurred or places which are closeby. The objective would be to create an awareness map, and determine why people react to an event. From that we would attempt to define an awareness metric. We want to see how factors other than physical proximity come into play such as country, culture, language, religion. With this we could determine which country has the highest level of international awareness. The project would require the Twitter API to acquire hashtag specific tweets with geolocation and therefore measure the awareness and reactions of different communities to a given event. GDELT would be used to recover standardised information regarding different events.
____
____
____


# 1. Recapitulation of Milestone 2

To recap what was done in the previous milestone, here we present a brief summary of the previous notebook which can be found here **(TODO : mettre le lien)**. 

____

## 1.1 Tweet Acquisition
A lot of time was spent acquiring the Tweets for two main reasons : 
- The Twitter dump did not contain the geolocations of the tweets or the users. 
- The Twitter API does not permit the recovery of tweets which are over 1 week old when searching for tweets with a given hashtag around a specific date. 

That is why a significant amount of time was put into recovering the tweets through webscraping using phantom js with random scrolling time and random breaks to avoid being blocked by Twitter.

____

## 1.2. Tweet Geolocalisation

Once the tweets were recovered we only kept those for which we had either the geolocalisation of the tweets itself or of the user. As the locations are manually input, they are not standardized and the corresponding location needed to be determined. 

The first step was therefore to format the strings by removing all special characters as setting everything to lower case for example.  Afterwards, we created multiple mappings which were used in the order presented as they represent the confidence in the given method : 
- A country mapper which uses multiple variations of the country name (different spellings and languages) to determine the country it corresponds to with the given ISO2 code
- A capital mapper which maps the capital of a country to the ISO2 code
- A first city mapper where duplicates were removed as there are multiple cities in the world with the same name (example : Torronto in Canada, the US and Australia). 
- A final city mapper with all the cities of all the countries in the world using the database provided here **(TODO : mettre le lien)**

## 1.3. Visualization of the Reaction For An Event

We then observed for one of the selected events the number of related tweets on a Chloropleth map. This helped us note certain issues in the mappings (high number of tweets in Antarctica for example). 

____


## 1.4. Enriching the Data 

To create our awareness model we wanted to recover information which could be used to determine a sort of distance between the different countries, that is why we recovered for all countries : 
- population
- area
- GDP
- languages
- religions
- government types
- border countries
- latitude and longitude

____
____
____


# 2. Creating the Awareness Model

To create our model we decided to construct a graph linking all countries where the weights of the segments would represent the proximity between countries in terms of awareness to major events. 

For that we used the dataset containing all the information regarding the countries and looked to give numeric values to all the different categories mentioned previously and a few new ones: 

1. Languages
2. Governments 
3. Distances between countries :
    - Birds eye view distances 
    - Hop matrix : the number of countries which separate two given countries (**TODO : noter quil y a le souci des distances a traverser au sein des pays qui n'est pas pris en compte, exemple de la Finlande a la Chine qui passe uniquement par la russie**). 
    - Normalized Adj matrix : a matrix which gives the neighboring countries with a normalizing factor to account for multiple countries on the border
    - Flight routes between countries
4. Religion
5. Population, Area, GDP

What needed to be taken into account here is that we have many variables which are categoric. Therefore we needed to find a way to attribute numeric values in order to construct the final graph.


**TODO : voir si le fait que le pays soit très grand avec une grande population qui peut diminuer l'intérêt par rapport à ce qui se passe ailleurs dans le monde**

____

## 2.1. Languages

Languages are representative of culture and most of the time, countries which speak the same language have a given proximity. That is why this metric was used to create the graph. 

Two main possibilities presented themselves :
- binary : if two countries have the same official language then they are linked
- more informed : some languages are similar in the sense of understanding the other language, learning it, etc. That is why we wanted to take into account Linguistic distances. Unfortunately this is a current research topic and there is no database which gives the Linguistic distances between all the official languages of all countries. Certain databases exist linking English to other languages but this is not sufficient for our application. That is why we decided to use another metric, albeit more abstact, which is to say the phylogenetic trees linking languages together. 


Using the phylogenetic trees taken here (TODO insert nice link) http://glottolog.org/glottolog/family, we were able to construct a distance metric and compute the distances between all languages as can be seen below. The notebook handling this can be found here (TODO : mettre le lien du notebook)

In [42]:
# TODO METTRE LA MATRICE DISTANCE ENTRE LES LANGUES

From this language distance matrix we were able to compute a standardized distance between all countries. 

The idea is the following :
- if countries have an official language in common, set the distance to 0
- else get the distances between the official languages and compute the average of the non infinite distances. 
- else set the distance to inf


In [43]:
# TODO METTRE LA MATRICE DISTANCE ENTRE LES PAYS SELON LA LANGUE

All of these values are then standardized. 

In [44]:
# TODO : METTRE LA MATRICE STANDARDIZE

____

## 2.2. Governments

The type of government is also a good representative of the people's perception to worldwide events. People of different countries with the same government type will be influenced in a similar way. 

After getting the list of all the government types (see Country data gathering.ipynb for more details on how we got them) we have to convert them to a numeric scale with a meaning.

First, we start by dividing all the government types in three groups and assigning them numerical values -1, 0 and 1. These are the resulting groups:
- Group 1:
    - 'parliamentary democracy'
    - 'parliamentary republic'
    - 'federal republic'
    - 'federation of monarchies'
    - 'semi-presidential republic'
    - 'semi-presidential federation'
- Group 2: 
    - 'non-self-governing overseas territory'
    - 'in transition'
    - 'unknown'
- Group 3: 
    - 'presidential republic'
    - 'presidential democracy'
    - 'monarchy'
    - 'theocratic republic'
    - 'communist state'
    - 'absolute monarchy'

As our criteria to group the government types, we used the variable of the power that the leaders of the government have. Coutries where the leader has a lot of power go in group 3 and will be assigned to the value of 1, the others in group 1 and they will be assigned the value -1. For the non well defined government types, we placed them at the center of the scale with the value of 0.

Here is the actual implementation (full code on Dataframes.ipynb):

In [45]:
data = pd.read_pickle('.\DataEnriching\data.pickle')
gov_type_array = [['parliamentary democracy', 'parliamentary republic', 'federal republic', 'federation of monarchies', \
                  'semi-presidential republic', 'semi-presidential federation'], 
                  ['non-self-governing overseas territory', 'in transition', 'unknown'], 
                  ['presidential republic', 'presidential democracy', 'monarchy', 'theocratic republic', 'communist state',\
                  'absolute monarchy']]
mapping_gov_type=dict()
for i, gov_group in enumerate(gov_type_array):
    for gov in gov_group:
        if i==0:
            mapping_gov_type.update({gov: -1})
        elif i==1:
            mapping_gov_type.update({gov: 0})
        else:
            mapping_gov_type.update({gov: 1})

# Mapping gov_type values to their numerical value (1, 0, -1)
data['gov_type_num'] = data.gov_type.map(mapping_gov_type)
data[['gov_type', 'gov_type_num']].head()

,gov_type,gov_type_num
name,,
Aruba,parliamentary democracy,-1.0
Afghanistan,presidential republic,1.0
Angola,presidential republic,1.0
Anguilla,parliamentary democracy,-1.0
Åland Islands,unknown,0.0


____

## 2.3. Distances 

____

## 2.4. Religions 

Religion, like government, affects the way of thinking of the people. Religion bonds between countries will definitely have an impact on the awareness of events that happen in countries with the same religions.

By reading the raw data excel file, we directly have the data in the format that is needed so we just selected the neccessary columns. Each column contains the percentage of adherents of the religion in each country.

Here is the dataframe with all the selected data (full code on Dataframes.ipynb):

In [46]:
rel_df = pd.read_pickle(".\DataEnriching\Pickles for Milestone 3\\final_rel_df.pickle")
rel_df.head()

name     religion                              \
                           Christianism Judaism     Islam  Buddhism   
ISO2                                                                  
AF             Afghanistan       0.0003     0.0  0.995600  0.000099   
AO                  Angola       0.8912     0.0  0.010399  0.000099   
AL                 Albania       0.2144     0.0  0.630000  0.000000   
AD                 Andorra       0.9070     0.0  0.008999  0.000000   
AE    United Arab Emirates       0.0714     0.0  0.674800  0.003500   

                                                                              \
     Zoroastrian   Hindu      Sikh Shinto    Baha'i Taoism Jain Confucianism   
ISO2                                                                           
AF      0.000099  0.0003  0.000099    0.0  0.000099    0.0  0.0          0.0   
AO      0.000000  0.0000  0.000000    0.0  0.000099    0.0  0.0          0.0   
AL      0.000000  0.0000  0.000000    0.0  0.002200    0.0  0.0          0.0   
AD      0.000000  0.0035  0.000000    0.0  0.000000    0.0  0.0          0.0   
AE      0.000000  0.2225  0.000000    0.0  0.009999    0.0  0.0          0.0   

                                                                          
     Syncretic religions Animist religions Non-religious Other religions  
ISO2                                                                      
AF                   0.0          0.000099      0.002000          0.0014  
AO                   0.0          0.075899      0.017899          0.0044  
AL                   0.0          0.000000      0.150700          0.0027  
AD                   0.0          0.000000      0.079600          0.0009  
AE                   0.0          0.000000      0.013599          0.0041

____

## 2.5. Population, Area, GDP 

This data is also straight forward to extract. From the data.pickle file we can extract it directly. The result is the following:

In [47]:
area_POP_gdp_df = pd.read_pickle('.\DataEnriching\Pickles for Milestone 3\\area_POP_gdp_df.pickle')
area_POP_gdp_df.head(10)

,area,POP,2016_gdp_capita
ISO2,,,
AW,180.0,NaN,NaN
AF,652230.0,27000000.0,561.778746
AO,1246700.0,19114176.0,3110.808183
AI,91.0,NaN,NaN
AX,1580.0,NaN,NaN
AL,28748.0,3195525.0,4146.896250
AD,468.0,85500.0,NaN
AE,83600.0,6236650.0,37622.207458
AR,2780400.0,40399992.0,12449.216852


____

## 2.6. Constructing the Graph

____
____
____

# 3. Improving the Mappings

____
____
____

# 4. Predictions Using the Awareness Model and Graph Diffusion